In [1]:
import os
import re
import ast
import torch
import json
from datetime import datetime
from PyPDF2 import PdfReader

from pydantic import BaseModel, Field, validator
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from transformers import AutoModelForCausalLM, AutoTokenizer

c:\Users\citak\anaconda3\envs\pytorchv2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import outlines
from enum import Enum
from langchain_community.llms import Outlines

from pydantic import BaseModel

In [3]:
model_id = "Qwen/Qwen2.5-3B-Instruct" # HuggingFaceTB/SmolLM2-135M-Instruct, HuggingFaceTB/SmolLM2-1.7B-Instruct, Qwen/Qwen2.5-3B-Instruct

llm = outlines.from_transformers(
    AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", torch_dtype=torch.bfloat16),
    AutoTokenizer.from_pretrained(model_id)
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.89s/it]


In [30]:
class Country(str, Enum):
    usa = "USA"
    turkey = "Turkey"
    germany = "Germany"
    france = "France"


class MovieReview(BaseModel):
    title: str = Field(description="The name of the movie", example="Inception")
    rating: int = Field(ge=1, le=10, description="Rating between 1 and 10")
    summary: str = Field(description="A short description of the movie plot", max_length=200)
    country: Country = Field(description="Country where the movie was produced")
    year: int = Field(ge=1900, le=2025, description="Release year of the movie")

In [31]:
movie_name = "Requiem For A Dream"

prompt = f"""
<|system|>
You are a helpful travel assistant.<|end|>
<|user|>
Write a short review for the movie '{movie_name}'<|end|>.
<|assistant|>
"""

In [39]:
short_review = llm(
    prompt,
    MovieReview,
    max_new_tokens=256,
    temperature=0.7,           # Controls randomness (lower = more deterministic)
    repetition_penalty=1.0     # Penalizes repeated phrases/tokens
    )

print(short_review)

{"title":"Requiem for a Dream: A Cinematic Tragedy","rating":5,"summary":"Requiem for a Dream is a harrowing cinematic experience that vividly captures the devastating impact of drug addiction on the human psyche. Directed by Darren Aronofsky, this film is a stark portrayal", "country":"USA","year":2000}


In [40]:
# Use structured data to route the MovieReview
def alert_manager(short_review):
    print("Alert!", short_review)


short_review = MovieReview.model_validate_json(short_review)

print(short_review)

title='Requiem for a Dream: A Cinematic Tragedy' rating=5 summary='Requiem for a Dream is a harrowing cinematic experience that vividly captures the devastating impact of drug addiction on the human psyche. Directed by Darren Aronofsky, this film is a stark portrayal' country=<Country.usa: 'USA'> year=2000


In [41]:
short_review.title, short_review.rating, short_review.country, short_review.year

('Requiem for a Dream: A Cinematic Tragedy', 5, <Country.usa: 'USA'>, 2000)

In [38]:
short_review.summary

"Requiem For A Dream is a harrowing and intense film that explores the dark side of addiction through the lives of four individuals. The movie's cinematography and soundtrack effectively convey the raw"